In [1]:
try:    
    import torch
    import torchvision
    assert int(torchvision.__version__.split(".")[1]) > 13
    print(torch.__version__, torchvision.__version__)
    print(f"imported torch version: {torch.__version__}, torchvision version {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required")


2.0.1+cu118 0.15.2+cu118
imported torch version: 2.0.1+cu118, torchvision version 0.15.2+cu118


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
def set_seed(seed: int=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [4]:
from pathlib import Path

Path("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip").name

'pizza_steak_sushi.zip'

### get data | test
----| ----|

In [5]:
from pathlib import Path

source = Path("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
data_path = Path("data")
image_path = data_path / "pizza_steak_sushi"
target_file = Path(source).name
download_path = data_path / target_file
download_path.is_file()

False

In [6]:
from pathlib import Path
import requests, zipfile, os

###modified version
def download_data(source: str,
                 destination: str,
                 remove_source: bool = True) ->Path:
    
    data_path = Path("data")
    image_path = data_path / destination
    target_file = Path(source).name
    download_path = data_path / target_file
    
    if image_path.is_dir():
        print(f"file already exist")
    elif not download_path.is_file():
        image_path.mkdir(parents=True, exist_ok=True)        
        with open(data_path / target_file, "wb") as f:
            request = requests.get(source)
            print(f"Downloading {target_file} from {source}...")
            f.write(request.content)

        print(f"extracting {target_file} to {image_path}")
        with zipfile.ZipFile(data_path / target_file, "r") as zip_ref:
            zip_ref.extractall(image_path)
            
        if remove_source:
            os.remove(data_path / target_file)
    else:
        print(f"extracting {target_file} to {image_path}")
        with zipfile.ZipFile(data_path / target_file, "r") as zip_ref:
            zip_ref.extractall(image_path)
            
        if remove_source:
            os.remove(data_path / target_file)
            
    return image_path


download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
             destination="pizza_steak_sushi")

file already exist


WindowsPath('data/pizza_steak_sushi')

In [7]:
# !pip install -U torchvision

In [8]:
from going_modular.going_modular import data_setup, engine

train_dir = image_path / "train"
test_dir = image_path / "test"
BATCH_SIZE = 32
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
auto_transform = weights.transforms()
print(f"auto_transforms: {auto_transform}")

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir,
                                                                                test_dir,
                                                                                auto_transform,
                                                                                BATCH_SIZE)

train_dataloader, test_dataloader, class_names

auto_transforms: ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)


(<torch.utils.data.dataloader.DataLoader at 0x2bc8829cd60>,
 ['pizza', 'steak', 'sushi'])

In [9]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
model = torchvision.models.efficientnet_b0(weights=weights).to(device)

In [10]:
for name, param in model.named_parameters():
    print(name)

features.0.0.weight
features.0.1.weight
features.0.1.bias
features.1.0.block.0.0.weight
features.1.0.block.0.1.weight
features.1.0.block.0.1.bias
features.1.0.block.1.fc1.weight
features.1.0.block.1.fc1.bias
features.1.0.block.1.fc2.weight
features.1.0.block.1.fc2.bias
features.1.0.block.2.0.weight
features.1.0.block.2.1.weight
features.1.0.block.2.1.bias
features.2.0.block.0.0.weight
features.2.0.block.0.1.weight
features.2.0.block.0.1.bias
features.2.0.block.1.0.weight
features.2.0.block.1.1.weight
features.2.0.block.1.1.bias
features.2.0.block.2.fc1.weight
features.2.0.block.2.fc1.bias
features.2.0.block.2.fc2.weight
features.2.0.block.2.fc2.bias
features.2.0.block.3.0.weight
features.2.0.block.3.1.weight
features.2.0.block.3.1.bias
features.2.1.block.0.0.weight
features.2.1.block.0.1.weight
features.2.1.block.0.1.bias
features.2.1.block.1.0.weight
features.2.1.block.1.1.weight
features.2.1.block.1.1.bias
features.2.1.block.2.fc1.weight
features.2.1.block.2.fc1.bias
features.2.1.blo

In [11]:
from torch import nn

for param in model.features.parameters():
    param.requires_grad = False
    
set_seed(42)

model.classifier = nn.Sequential(
                                 nn.Dropout(p=0.8,inplace=True),
                                 nn.Linear(in_features=1280,
                                          out_features=len(class_names),
                                          bias=True)
                                )


In [12]:
from torchinfo import summary

summary(model,
   input_size=(32,3,224,244),
       col_names = ["input_size","output_size","num_params","trainable"],
       col_width=15,
       row_settings=["var_names"],
       verbose= 0)


Layer (type (var_name))                                      Input Shape     Output Shape    Param #         Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 244] [32, 3]         --              Partial
├─Sequential (features)                                      [32, 3, 224, 244] [32, 1280, 7, 8] --              False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 244] [32, 32, 112, 122] --              False
│    │    └─Conv2d (0)                                       [32, 3, 224, 244] [32, 32, 112, 122] (864)           False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 122] [32, 32, 112, 122] (64)            False
│    │    └─SiLU (2)                                         [32, 32, 112, 122] [32, 32, 112, 122] --              --
│    └─Sequential (1)                                        [32, 32, 112, 122] [32, 16, 112, 122] --              False
│    │    └─MBConv (0)                      

In [13]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), 0.001)

In [14]:
try:
    from torch.utils.tensorboard import SummaryWriter
except ModuleNotFoundError:
    !pip install tensorboard
    from torch.utils.tensorboard import SummaryWriter

In [67]:
from torch.utils.tensorboard import SummaryWriter
# (data_path/"runs").mkdir(parents=True,exist_ok=True)
# writer = SummaryWriter()

In [16]:
# from going_modular.going_modular.engine import train_step,test_step
# from tqdm.auto import tqdm
# from collections import defaultdict

# def train(model,
#          train_data_loader,
#           test_data_loader,
#          loss_fn,
#          optimizer,
#          device,
#          epochs = 5):
    
#     results = defaultdict(list)
    
#     for epoch in tqdm(range(epochs)):
#         train_loss, train_acc = train_step(model,
#                   train_dataloader,
#                   loss_fn,
#                   optimizer,
#                   device)

#         test_loss,test_acc  = test_step(model,
#                  test_dataloader,
#                  loss_fn,
#                  device)
        
#         print(
#           f"Epoch: {epoch+1} | "
#           f"train_loss: {train_loss:.4f} | "
#           f"train_acc: {train_acc:.4f} | "
#           f"test_loss: {test_loss:.4f} | "
#           f"test_acc: {test_acc:.4f}"
#         )
        
#         results["train_loss"].append(train_loss)
#         results["train_acc"].append(train_acc)
#         results["test_loss"].append(test_loss)
#         results["test_acc"].append(test_acc)
        
#         ##Experiment tracking
#         writer.add_scalars(main_tag="Loss",
#                         tag_scalar_dict={"train_loss":train_loss,"test_loss":test_loss},
#                           global_step=epoch) 
        
#         writer.add_scalars(main_tag="Accuracy",
#                           tag_scalar_dict={"train_acc":train_acc,"test_acc":test_acc},
#                           global_step=epoch)
        
#         writer.add_graph(model=model,
#                          ## pass in an example input
#                         input_to_model=torch.randn(32,3,224,224).to(device))
        
#     writer.close()
    
#     return results
         

In [17]:
# set_seed()
# results = train(model,
#                train_dataloader,
#                test_dataloader,
#                loss_fn,
#                optimizer,
#                device)

In [18]:
##practice
# from going_modular.going_modular import engine_with_tensorboard

# results = engine_with_tensorboard.train(model,
#                               train_dataloader,
#                               test_dataloader,
#                               loss_fn,
#                               optimizer,
#                               device,
#                               epochs=5
#                              )

In [19]:
# results

In [20]:
import sys 
sys.path
# sys.executable
# !pip3 show tensorboard

['C:\\Users\\pouya\\jupyter projects\\PyTorch_for Deep Learning_Machine Learning',
 'C:\\ProgramData\\Anaconda3\\python39.zip',
 'C:\\ProgramData\\Anaconda3\\DLLs',
 'C:\\ProgramData\\Anaconda3\\lib',
 'C:\\ProgramData\\Anaconda3',
 '',
 'C:\\Users\\pouya\\AppData\\Roaming\\Python\\Python39\\site-packages',
 'C:\\ProgramData\\Anaconda3\\lib\\site-packages',
 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32',
 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\Pythonwin']

In [21]:
# %load_ext tensorboard
# %tensorboard --logdir runs --host localhost

In [22]:
# pip install datetime

In [23]:
# def create_writer(experiment_name,
#                  model_name,
#                  extra=None):
#     """ create a custom writer
    
#     Args:
#         experiment name
#         model
#         extra
        
#     Returns:
#         torch.utils.tensorboard.writer.SummaryWriter()
        
#     """
    
#     from datetime import datetime as dt
#     import os
#     from pathlib import Path
    
# #     now = dt.now().strftime("%Y-%m-%d")
#     now = dt.now().strftime("%b %d-%Y")
    
#     if extra:
# #         log_dir = os.path.join("runs",now, experiment_name, model_name, extra)
#         log_dir = Path("runs")/now/ experiment_name/ model_name/ extra
        
#     else:
# #         log_dir = os.path.join("runs",now, experiment_name, model_name, extra)
#         log_dir = Path("runs")/now/ experiment_name/ model_name
        
#     print(f"saving log to {log_dir}")
#     return SummaryWriter(log_dir=log_dir)
                

In [24]:
# create_writer("data_10%",
#              "efficientnetb0",
#              "5_epochs")

In [25]:
# %load_ext tensorboard
# %tensorboard --logdir log_dir

In [26]:
from going_modular.going_modular.engine import train_step, test_step
from time import perf_counter
from tqdm.auto import tqdm


def train(model,
         train_dataloader,
         test_loader,
         optimizer,
         loss_fn,
         device,
         epochs,
         writer,
         ):
    
    results = {"train_loss": [],
              "test_loss":[],
              "train_acc":[],
              "test_acc":[]}
    
    train_start_time = perf_counter()
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model,
                                          train_dataloader,
                                          loss_fn,
                                          optimizer,
                                          device)
        
        test_loss, test_acc = test_step(model,
                                       test_dataloader,
                                       loss_fn,
                                       device
                                       )
        
        print( f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"       
        )
        
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        
        if writer:
            writer.add_scalars(main_tag="loss",
                              tag_scalar_dict={"train_loss":train_loss,
                                              "test_loss":test_loss},
                              global_step=epoch)
            
            writer.add_scalars(main_tag="accuracy",
                              tag_scalar_dict={"train_acc":train_acc,
                                              "test_acc":test_acc},
                              global_step=epoch)
            
                
            writer.close()
        else:
            pass
            
    train_end_time = perf_counter()   
    
    print(f"total train time took {train_end_time-train_start_time} on {device}")
    
    return results

In [27]:
# set_seed()

# writer = create_writer("data_10%_customWriter",
#              "efficientnetb0",
#              "4_epochs")
# results = train(model,
#                train_dataloader,
#                test_dataloader,     
#                optimizer,
#                 loss_fn,
#                device,
#                 4,
#                 writer,
#                )

Breaking these down we get: 

| Experiment number | Training Dataset | Model (pretrained on ImageNet) | Number of epochs |
| ----- | ----- | ----- | ----- |
| 1 | Pizza, Steak, Sushi 10% percent | EfficientNetB0 | 5 |
| 2 | Pizza, Steak, Sushi 10% percent | EfficientNetB2 | 5 | 
| 3 | Pizza, Steak, Sushi 10% percent | EfficientNetB0 | 10 | 
| 4 | Pizza, Steak, Sushi 10% percent | EfficientNetB2 | 10 |
| 5 | Pizza, Steak, Sushi 20% percent | EfficientNetB0 | 5 |
| 6 | Pizza, Steak, Sushi 20% percent | EfficientNetB2 | 5 |
| 7 | Pizza, Steak, Sushi 20% percent | EfficientNetB0 | 10 |
| 8 | Pizza, Steak, Sushi 20% percent | EfficientNetB2 | 10 |

In [28]:
###create download data function again

def download_data(source,
                 destination,
                  remove_source=True
                 ):
    
    data_path = Path("data")
    image_path = data_path / destination
    
    if image_path.is_dir():
        print("already exist")
    else:
        print(f"{image_path} does not exist,downloading...")
        target_dir = Path(source).name
        image_path.mkdir(parents=True,exist_ok=True)
        with open(data_path/target_dir, "wb") as f:
            response = requests.get(source)
            f.write(response.content)
            
        with zipfile.ZipFile(data_path/target_dir, "r") as zip_ref:
            zip_ref.extractall(image_path)
            
        if remove_source:
            os.remove(data_path/target_dir)
            
    return image_path


In [29]:
data_10_percent_path = download_data(source= r"https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
             destination="pizza_steak_sushi")

data_20_percent_path = download_data(source=r"https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip",
                                    destination="pizza_steak_sushi_20_percent")

already exist
already exist


In [30]:
train_dir_10_percent = data_10_percent_path / "train"
train_dir_20_percent = data_20_percent_path / "train"

##we keep test dir same for both of our tests
test_dir = data_10_percent_path / "test"

train_dir_10_percent,train_dir_20_percent,test_dir

(WindowsPath('data/pizza_steak_sushi/train'),
 WindowsPath('data/pizza_steak_sushi_20_percent/train'),
 WindowsPath('data/pizza_steak_sushi/test'))

In [31]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

simple_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
])

In [32]:
train_data_10_percent = datasets.ImageFolder(root=train_dir_10_percent,
                                            transform=simple_transform)

train_data_20_percent = datasets.ImageFolder(train_dir_20_percent,
                                            simple_transform)

test_data_10_percent = datasets.ImageFolder(test_dir,
                                           simple_transform)


In [33]:
BATCH_SIZE = 32

train_dataloader_10_percent = DataLoader(train_data_10_percent,
                                        BATCH_SIZE,
                                        True,
                                        pin_memory=True)

train_dataloader_20_percent = DataLoader(train_data_20_percent,
                                        BATCH_SIZE,
                                        True,
                                        pin_memory=True)

test_dataloader_10_percent = DataLoader(test_data_10_percent,
                                        BATCH_SIZE,
                                        True,
                                       pin_memory=True)

class_names = train_data_10_percent.classes

print(f"Number of batches of size {BATCH_SIZE} in 10 percent training data: {len(train_dataloader_10_percent)}")
print(f"Number of batches of size {BATCH_SIZE} in 20 percent training data: {len(train_dataloader_20_percent)}")
print(f"Number of batches of size {BATCH_SIZE} in testing data: {len(train_dataloader_10_percent)} (all experiments will use the same test set)")
print(f"Number of classes: {len(class_names)}, class names: {class_names}")

Number of batches of size 32 in 10 percent training data: 8
Number of batches of size 32 in 20 percent training data: 15
Number of batches of size 32 in testing data: 8 (all experiments will use the same test set)
Number of classes: 3, class names: ['pizza', 'steak', 'sushi']


In [34]:
effnetb2_weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT

effnetb2_model = torchvision.models.efficientnet_b0(weights=weights).to(device)

summary(effnetb2_model,
        input_size=(32, 3, 224, 224),
       col_names=["input_size","output_size","num_params","trainable"],
       col_width=15,
       row_settings=["var_names"] 
       )

Layer (type (var_name))                                      Input Shape     Output Shape    Param #         Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224] [32, 1000]      --              True
├─Sequential (features)                                      [32, 3, 224, 224] [32, 1280, 7, 7] --              True
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224] [32, 32, 112, 112] --              True
│    │    └─Conv2d (0)                                       [32, 3, 224, 224] [32, 32, 112, 112] 864             True
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112] [32, 32, 112, 112] 64              True
│    │    └─SiLU (2)                                         [32, 32, 112, 112] [32, 32, 112, 112] --              --
│    └─Sequential (1)                                        [32, 32, 112, 112] [32, 16, 112, 112] --              True
│    │    └─MBConv (0)                              

In [35]:
import torchvision
from torch import nn

OUT_FEATURES = len(class_names)

def create_effnetb0():
    weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
    model = torchvision.models.efficientnet_b0(weights=weights)
    
    for param in model.features.parameters():
        param.requires_grad = False
    
    set_seed()
    
    model.classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=1280, out_features=OUT_FEATURES)
    ).to(device)
    
    model.name = "effnetb0"
    
    print(f"created {model.name} model")
    
    return model   

def create_effnetb2():
    weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
    model = torchvision.models.efficientnet_b2(weights=weights).to(device)

    for param in model.features.parameters():
        param.requires_grad = False

    set_seed()


    model.classifier = nn.Sequential(
        nn.Dropout(p=0.3),
        nn.Linear(in_features=1408, out_features=OUT_FEATURES)
    ).to(device)

    model.name = "effnetb2"
    print(f"Created {model.name} model.")
    return model

In [36]:
def get_summary(model):
    report = summary(model,
        input_size=(32, 3, 224, 224),
       col_names=["input_size","output_size","num_params","trainable"],
       col_width=15,
       row_settings=["var_names"] 
       )

            
    return report

In [37]:
effnetb0 = create_effnetb0()
effnetb2 = create_effnetb2()

get_summary(effnetb2)

created effnetb0 model
Created effnetb2 model.


Layer (type (var_name))                                      Input Shape     Output Shape    Param #         Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224] [32, 3]         --              Partial
├─Sequential (features)                                      [32, 3, 224, 224] [32, 1408, 7, 7] --              False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224] [32, 32, 112, 112] --              False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224] [32, 32, 112, 112] (864)           False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112] [32, 32, 112, 112] (64)            False
│    │    └─SiLU (2)                                         [32, 32, 112, 112] [32, 32, 112, 112] --              --
│    └─Sequential (1)                                        [32, 32, 112, 112] [32, 16, 112, 112] --              False
│    │    └─MBConv (0)                      

In [54]:
writer = SummaryWriter(Path("runs2"))

In [64]:
from going_modular.going_modular.engine import train_step, test_step
from time import perf_counter
from tqdm.auto import tqdm


def train(model,
         train_dataloader,
         test_loader,
         optimizer,
         loss_fn,
         device,
         epochs,
         writer,
          step,
         ):
    
    results = {"train_loss": [],
              "test_loss":[],
              "train_acc":[],
              "test_acc":[]}
    
    train_start_time = perf_counter()
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model,
                                          train_dataloader,
                                          loss_fn,
                                          optimizer,
                                          device)
        
        test_loss, test_acc = test_step(model,
                                       test_dataloader,
                                       loss_fn,
                                       device
                                       )
        
        print( f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"       
        )
        
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        
        if writer:
            writer.add_scalar(tag="training_loss",
                             scalar_value=train_loss,
                             global_step=step)
            
            writer.add_scalar("training accuracy",
                              train_acc,
                             global_step=step)
                            
            writer.close()
        else:
            pass
            
    train_end_time = perf_counter()   
    
    print(f"total train time took {train_end_time-train_start_time} on {device}")
    
    return results

In [58]:
num_epochs = [5,10]

models = ["effnetb0", "effnetb2"]

train_dataloaders = {"data_10_percent":train_dataloader_10_percent,
                    "data_20_percent":train_dataloader_20_percent}

In [39]:
##more practice
# def create_writer(experiment_name,
#                  model_name,
#                  extra=None):
    
#     now = dt.now().strftime("%d-%m-%Y")
#     if extra:
#         log_dir = Path("runs")/now /experiment_name/ model_name/extra
#     else:
#         log_dir = Path("runs")/now /experiment_name/ model_name
    
#     print(f"saving to {log_dir}")
#     SummaryWriter(log_dir=log_dir)
    

In [40]:
def create_writer(experiment_name: str, 
                  model_name: str, 
                  extra: str=None) -> torch.utils.tensorboard.writer.SummaryWriter():

    from datetime import datetime
    import os


    timestamp = datetime.now().strftime("%Y-%m-%d") # returns YYYY-MM-DD format

    if extra:
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name, extra)
    else:
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name)
        
    print(f"Created SummaryWriter, saving to: {log_dir}...")
    return SummaryWriter(log_dir=log_dir)

In [65]:
%%time
from datetime import datetime as dt

set_seed()
experiment_tracking = 0

for epochs in num_epochs:
    
    for model_name in models:
        
        for dataloader_name, train_dataloader in train_dataloaders.items():
            
            experiment_tracking += 1
            print(f"experiment number: {experiment_tracking}\nnumber of epochs: {epochs}\nmodel: {model_name}\ndata_loader:{dataloader_name}")
            
            if model_name == "effnetb0":
                model = create_effnetb0()
                
            else:
                model = create_effnetb2()
            
            model.to(device)
            loss_fn = torch.nn.CrossEntropyLoss()
            optimizer = torch.optim.Adam(params=model.parameters(),lr=0.001)
            
            train(model,
                  train_dataloader,
                  test_dataloader,
                  optimizer,
                  loss_fn,
                  device,
                  epochs,
                  writer,
                  experiment_tracking
#                   writer=create_writer(dataloader_name,
#                                       model_name,
#                                       extra=f"{epochs}_epochs")
                                
            )
            
            cur_time = dt.now().strftime("%b%d-%Y")
            
            saved_model_name = f"{model_name}_{dataloader_name}_{epochs}_epochs.pth"
            assert saved_model_name.endswith("pth") or saved_model_name.endswith("th")
            
            save_path = Path("Model")/cur_time/saved_model_name
            (Path("Model")/cur_time).mkdir(parents=True,exist_ok=True)
            
            torch.save(obj= model.state_dict(),
                      f= save_path)                
                    

experiment number: 1
number of epochs: 5
model: effnetb0
data_loader:data_10_percent
created effnetb0 model


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0520 | train_acc: 0.4961 | test_loss: 0.8505 | test_acc: 0.6496
Epoch: 2 | train_loss: 0.9251 | train_acc: 0.6055 | test_loss: 0.7662 | test_acc: 0.7017
Epoch: 3 | train_loss: 0.7707 | train_acc: 0.6992 | test_loss: 0.6602 | test_acc: 0.8759
Epoch: 4 | train_loss: 0.7085 | train_acc: 0.7617 | test_loss: 0.6719 | test_acc: 0.8059
Epoch: 5 | train_loss: 0.7050 | train_acc: 0.7773 | test_loss: 0.6078 | test_acc: 0.8759
total train time took 17.455327399999987 on cuda
experiment number: 2
number of epochs: 5
model: effnetb0
data_loader:data_20_percent
created effnetb0 model


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.9588 | train_acc: 0.6125 | test_loss: 0.6506 | test_acc: 0.8759
Epoch: 2 | train_loss: 0.6833 | train_acc: 0.8438 | test_loss: 0.5657 | test_acc: 0.9072
Epoch: 3 | train_loss: 0.5729 | train_acc: 0.8792 | test_loss: 0.4881 | test_acc: 0.9072
Epoch: 4 | train_loss: 0.5320 | train_acc: 0.8271 | test_loss: 0.4978 | test_acc: 0.8570
Epoch: 5 | train_loss: 0.4486 | train_acc: 0.8833 | test_loss: 0.4077 | test_acc: 0.9176
total train time took 30.10120460000053 on cuda
experiment number: 3
number of epochs: 5
model: effnetb2
data_loader:data_10_percent
Created effnetb2 model.


...

Epoch: 1 | train_loss: 0.9588 | train_acc: 0.6125 | test_loss: 0.6506 | test_acc: 0.8759
Epoch: 2 | train_loss: 0.6833 | train_acc: 0.8438 | test_loss: 0.5657 | test_acc: 0.9072
Epoch: 3 | train_loss: 0.5729 | train_acc: 0.8792 | test_loss: 0.4881 | test_acc: 0.9072
Epoch: 4 | train_loss: 0.5320 | train_acc: 0.8271 | test_loss: 0.4978 | test_acc: 0.8570
Epoch: 5 | train_loss: 0.4486 | train_acc: 0.8833 | test_loss: 0.4077 | test_acc: 0.9176
Epoch: 6 | train_loss: 0.4670 | train_acc: 0.8438 | test_loss: 0.3895 | test_acc: 0.8977
Epoch: 7 | train_loss: 0.3625 | train_acc: 0.9062 | test_loss: 0.3452 | test_acc: 0.8968
Epoch: 8 | train_loss: 0.3405 | train_acc: 0.9125 | test_loss: 0.3040 | test_acc: 0.9167
Epoch: 9 | train_loss: 0.3388 | train_acc: 0.9021 | test_loss: 0.3982 | test_acc: 0.8883
Epoch: 10 | train_loss: 0.3613 | train_acc: 0.8521 | test_loss: 0.3127 | test_acc: 0.9271
total train time took 58.93437200000062 on cuda
experiment number: 7
number of epochs: 10
model: effnetb2
dat

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0914 | train_acc: 0.3867 | test_loss: 0.9381 | test_acc: 0.6799
Epoch: 2 | train_loss: 0.8953 | train_acc: 0.6719 | test_loss: 0.8351 | test_acc: 0.8447
Epoch: 3 | train_loss: 0.8227 | train_acc: 0.6914 | test_loss: 0.7777 | test_acc: 0.8352
Epoch: 4 | train_loss: 0.7385 | train_acc: 0.7812 | test_loss: 0.7414 | test_acc: 0.8153
Epoch: 5 | train_loss: 0.7253 | train_acc: 0.7617 | test_loss: 0.7075 | test_acc: 0.8258
Epoch: 6 | train_loss: 0.6059 | train_acc: 0.7852 | test_loss: 0.6875 | test_acc: 0.7955
Epoch: 7 | train_loss: 0.5351 | train_acc: 0.9141 | test_loss: 0.6133 | test_acc: 0.9062
Epoch: 8 | train_loss: 0.5073 | train_acc: 0.9219 | test_loss: 0.6103 | test_acc: 0.8864
Epoch: 9 | train_loss: 0.5477 | train_acc: 0.7812 | test_loss: 0.6468 | test_acc: 0.8561
Epoch: 10 | train_loss: 0.4857 | train_acc: 0.9336 | test_loss: 0.5311 | test_acc: 0.8864
total train time took 38.21960709999985 on cuda
experiment number: 8
number of epochs: 10
model: effnetb2
dat

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.9819 | train_acc: 0.5604 | test_loss: 0.7578 | test_acc: 0.8551
Epoch: 2 | train_loss: 0.7309 | train_acc: 0.8042 | test_loss: 0.6631 | test_acc: 0.8561
Epoch: 3 | train_loss: 0.5988 | train_acc: 0.8521 | test_loss: 0.5708 | test_acc: 0.8958
Epoch: 4 | train_loss: 0.4859 | train_acc: 0.9083 | test_loss: 0.5246 | test_acc: 0.8769
Epoch: 5 | train_loss: 0.4775 | train_acc: 0.8354 | test_loss: 0.5045 | test_acc: 0.8466
Epoch: 6 | train_loss: 0.4247 | train_acc: 0.8750 | test_loss: 0.4793 | test_acc: 0.8466
Epoch: 7 | train_loss: 0.3475 | train_acc: 0.9437 | test_loss: 0.4006 | test_acc: 0.9062
Epoch: 8 | train_loss: 0.3321 | train_acc: 0.9375 | test_loss: 0.3979 | test_acc: 0.8864
Epoch: 9 | train_loss: 0.3371 | train_acc: 0.8938 | test_loss: 0.4662 | test_acc: 0.8665
Epoch: 10 | train_loss: 0.3599 | train_acc: 0.8875 | test_loss: 0.3828 | test_acc: 0.8968
total train time took 65.82697170000029 on cuda
CPU times: total: 19min 55s
Wall time: 5min 3s


In [66]:
# %load_ext tensorboard
%tensorboard --logdir runs2 

In [53]:
# %load_ext tensorboard
# %tensorboard --logdir runs 

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 14560), started 0:00:52 ago. (Use '!kill 14560' to kill it.)

In [ ]:
###in case of tensorboard error
# taskkill /im tensorboard.exe /f
# del /q %TMP%\.tensorboard-info\*
# C:\Users\pouya\AppData\Local\Temp\.tensorboard-info


In [ ]:
best_model_path = "Model/Sep11-2023/effnetb2_data_20_percent_10_epochs.pth"

best_model = create_effnetb2()

best_model.load_state_dict(torch.load(best_model_path))

In [ ]:
####practice

# from torchmetrics import ConfusionMatrix
# from mlxtend.plotting import plot_confusion_matrix 

# confmat = ConfusionMatrix(num_classes=len(class_names),task="multiclass")
# confmat_tensor = confmat(y_pred,
#                         test_data_10_percent.targets)

# ax, fig = plot_confusion_matrix(confmat_tensor.numpy(),
#                                class_names=class_names,
#                                figsize=(15,7))


In [ ]:
 f"effnetb2: {Path(best_model_path).stat().st_size // (1024*1024)} Mb"

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

def pred_and_plot(model,
                 img_path,
                  class_names,
                  img_size = (224, 224),
                 transform = None,
                 ):
    
    img = Image.open(img_path)
    
    if transform:
        img_transform = transform(img)
    else:
        img_transform = transforms.Compose([
            transforms.Resize(img_size),
            transforms.ToTensor(),
            transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),                             
        ])
        
    model.to(device)
    
    with torch.inference_mode():
        transformed_img = img_transform(img).unsqueeze(0)
        targ_img_pred = model(transformed_img.to(device))
        
    targ_img_prob = torch.softmax(targ_img_pred,dim=1)
    targ_img_label = torch.argmax(targ_img_prob,dim=1)
    
    plt.figure()
    plt.imshow(img)
    plt.title(f"pred: {class_names[targ_img_label]}| prob: {targ_img_prob.max()}")
    plt.axis(False);

In [ ]:
import random

test_dir_20_percent_path_str = "data/pizza_steak_sushi_20_percent/test"

test_img_path_list = list(Path(test_dir_20_percent_path_str).glob("*/*.jpg"))
test_img_path_samble = random.sample(test_img_path_list, k=3)

for path in test_img_path_samble:
    pred_and_plot(best_model,
                  path,
                  class_names
                 )

In [ ]:
import requests

custom_img_path = Path("data/04-pizza-dad.jpg")

if not custom_img_path.is_file():
    with open(custom_img_path,"wb") as f:
        request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/images/04-pizza-dad.jpeg")
        f.write(request.content)        
else:
    print(f"skipping download,{custom_img_path} already exist")
    
pred_and_plot(best_model,
              custom_img_path,
              class_names
             )
    

In [ ]:
best_model.eval()
with torch.inference_mode():
    y_pred_labels = []
    for X, y in test_dataloader_10_percent:
        X, y = X.to(device), y.to(device)
        
        y_pred = best_model(X)
        y_pred_prob = torch.softmax(y_pred,dim=1)
        y_pred_label = torch.argmax(y_pred_prob,dim=1)
        y_pred_labels.append(y_pred_label.cpu())
        
    torch.cat(y_pred_labels)
        
pred_labels_tensor =  torch.cat(y_pred_labels)      
pred_labels_tensor

In [ ]:
truth_labels = [y for X,y in test_dataloader_10_percent]
truth_labels = torch.cat(truth_labels)
truth_labels

In [ ]:
torch.tensor(test_data_10_percent.targets)

In [ ]:
###practice

from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix 

confmat = ConfusionMatrix(num_classes=len(class_names),task="multiclass")
confmat_tensor = confmat(preds=pred_labels_tensor,
                        target=truth_labels)   ##or use torch.tensor(test_data_10_percent.targets)

ax, fig = plot_confusion_matrix(confmat_tensor.numpy(),
                               class_names=class_names,
                               figsize=(10,7))
